In [1]:
import docker
import uuid

def run_in_master(command):
    client = docker.from_env()
    container = client.containers.get('master')
    return container.exec_run(command)

def get_data_from_output_path(path):
    return f"{path}/part-r-00000"

def print_hdfs_output(path):
    raw = run_in_master(f"hdfs dfs -cat {get_data_from_output_path(path)}").output.decode('utf-8')
    print(f"{raw[0:10000]}")
    


# Covid

## Wybór kolumn

In [2]:
covid_01_jar_path = "/data/master_volume/map_reduce_jars/covid_01.jar"
covid_01_input_path = "/datasets/covid-dataset.jsonl"
covid_01_output_path = "/out_covid_1" + str(uuid.uuid4())

res = run_in_master(f"yarn jar {covid_01_jar_path} {covid_01_input_path} {covid_01_output_path}")
print(res.exit_code == 0)

True


In [5]:
print_hdfs_output(covid_01_output_path)

id	{"date":"2023-04-19","country":"Zimbabwe","total_cases":264613,"new_cases":0,"total_deaths":264613,"new_deaths":0,"average_global_new_cases_per_million":160.28056}
id	{"date":"2023-04-18","country":"Zimbabwe","total_cases":264613,"new_cases":0,"total_deaths":264613,"new_deaths":0,"average_global_new_cases_per_million":160.28056}
id	{"date":"2023-04-17","country":"Zimbabwe","total_cases":264613,"new_cases":3,"total_deaths":264613,"new_deaths":0,"average_global_new_cases_per_million":160.28056}
id	{"date":"2023-04-16","country":"Zimbabwe","total_cases":264610,"new_cases":3,"total_deaths":264610,"new_deaths":0,"average_global_new_cases_per_million":160.28056}
id	{"date":"2023-04-15","country":"Zimbabwe","total_cases":264607,"new_cases":10,"total_deaths":264607,"new_deaths":0,"average_global_new_cases_per_million":160.28056}
id	{"date":"2023-04-14","country":"Zimbabwe","total_cases":264597,"new_cases":4,"total_deaths":264597,"new_deaths":0,"average_global_new_cases_per_million":160.2805

## Wybór wymiaru czasu

In [21]:
covid_02_jar_path = "/data/master_volume/map_reduce_jars/covid_02.jar"
covid_02_input_path = get_data_from_output_path(covid_01_output_path)
covid_02_output_path = "/out_covid_2" + str(uuid.uuid4())

res = run_in_master(f"yarn jar {covid_02_jar_path} {covid_02_input_path} {covid_02_output_path}")
print(res.exit_code == 0)

True


In [ ]:
print_hdfs_output(covid_02_output_path)

# Steam

## Agregacja danych ze źródeł Steam

In [3]:
steam_01_jar_path = "/data/master_volume/map_reduce_jars/steam_01_combine.jar"
steam_01_input_path = "/datasets/steam-dataset/steam_dataset/appinfo/store_data/steam_store_data.jsonl"
steam_01_input_path2 = "/datasets/steam-dataset/steam_dataset/steamspy/basic/steam_spy_scrap.jsonl"
steam_01_output_path = "/out_steam_1" + str(uuid.uuid4())

res = run_in_master(f"yarn jar {steam_01_jar_path} {steam_01_input_path} {steam_01_input_path2} {steam_01_output_path}")
print(res.exit_code == 0)

True


In [4]:
print_hdfs_output(steam_01_output_path)

10	{"game_id":10,"name":"Counter-Strike","positive":196557,"negative":5070,"owners":"10,000,000 .. 20,000,000","ccu":12877,"release_date":"2000-11-01"}
1000000	{"game_id":1000000,"name":"ASCENXION","positive":27,"negative":5,"owners":"0 .. 20,000","ccu":0,"release_date":"2021-05-14"}
1000010	{"game_id":1000010,"name":"Crown Trick","positive":3809,"negative":583,"owners":"200,000 .. 500,000","ccu":51,"release_date":"2020-10-16"}
1000030	{"game_id":1000030,"name":"Cook, Serve, Delicious! 3?!","positive":1469,"negative":101,"owners":"50,000 .. 100,000","ccu":54,"release_date":"2020-10-14"}
1000040	{"game_id":1000040,"name":"细胞战争","positive":0,"negative":1,"owners":"0 .. 20,000","ccu":0,"release_date":"2019-03-30"}
1000080	{"game_id":1000080,"name":"Zengeon","positive":1011,"negative":431,"owners":"50,000 .. 100,000","ccu":2,"release_date":"2019-06-24"}
1000100	{"game_id":1000100,"name":"干支セトラ　陽ノ卷｜干支etc.　陽之卷","positive":18,"negative":6,"owners":"0 .. 20,000","ccu":0,"release_date":"2019-01

## Selekcja gier

In [5]:
steam_02_jar_path = "/data/master_volume/map_reduce_jars/steam_02_choose.jar"
steam_02_input_path = get_data_from_output_path(steam_01_output_path)
steam_02_output_path = "/out_steam_2" + str(uuid.uuid4())

res = run_in_master(f"yarn jar {steam_02_jar_path} {steam_02_input_path} {steam_02_output_path}")
print(res.exit_code == 0)

True


In [6]:
print_hdfs_output(steam_02_output_path)

10	{"game_id":10,"name":"Counter-Strike","positive":196557,"negative":5070,"owners":"10,000,000 .. 20,000,000","ccu":12877,"release_date":"2000-11-01"}
105600	{"game_id":105600,"name":"Terraria","positive":941388,"negative":20346,"owners":"20,000,000 .. 50,000,000","ccu":39371,"release_date":"2011-05-16"}
1085660	{"game_id":1085660,"name":"Destiny 2","positive":396069,"negative":74899,"owners":"20,000,000 .. 50,000,000","ccu":106858,"release_date":"2019-10-01"}
218620	{"game_id":218620,"name":"PAYDAY 2","positive":513225,"negative":62272,"owners":"20,000,000 .. 50,000,000","ccu":39437,"release_date":"2013-08-13"}
227300	{"game_id":227300,"name":"Euro Truck Simulator 2","positive":535815,"negative":14706,"owners":"10,000,000 .. 20,000,000","ccu":33648,"release_date":"2012-10-12"}
230410	{"game_id":230410,"name":"Warframe","positive":437656,"negative":47546,"owners":"20,000,000 .. 50,000,000","ccu":50453,"release_date":"2013-03-25"}
240	{"game_id":240,"name":"Counter-Strike: Source","pos

## Wybór pierwszych N gier

In [7]:
steam_03_jar_path = "/data/master_volume/map_reduce_jars/steam_03_takeN.jar"
steam_03_input_path = get_data_from_output_path(steam_02_output_path)
steam_03_output_path = "/out_steam_3" + str(uuid.uuid4())

res = run_in_master(f"yarn jar {steam_03_jar_path} {steam_03_input_path} {steam_03_output_path}")
print(res.exit_code == 0)

True


In [8]:
print_hdfs_output(steam_03_output_path)

id	{"game_id":945360,"name":"Among Us"}
id	{"game_id":730,"name":"Counter-Strike: Global Offensive"}
id	{"game_id":582010,"name":"Monster Hunter: World"}
id	{"game_id":570,"name":"Dota 2"}
id	{"game_id":552990,"name":"World of Warships"}
id	{"game_id":550,"name":"Left 4 Dead 2"}
id	{"game_id":49520,"name":"Borderlands 2"}
id	{"game_id":444200,"name":"World of Tanks Blitz"}
id	{"game_id":444090,"name":"Paladins"}
id	{"game_id":440,"name":"Team Fortress 2"}
id	{"game_id":386360,"name":"SMITE"}
id	{"game_id":381210,"name":"Dead by Daylight"}
id	{"game_id":346110,"name":"ARK: Survival Evolved"}
id	{"game_id":322330,"name":"Don't Starve Together"}
id	{"game_id":304930,"name":"Unturned"}
id	{"game_id":292030,"name":"The Witcher 3: Wild Hunt"}
id	{"game_id":291550,"name":"Brawlhalla"}
id	{"game_id":271590,"name":"Grand Theft Auto V"}
id	{"game_id":252950,"name":"Rocket League"}
id	{"game_id":242760,"name":"The Forest"}



## Pobranie danych ze Steam Charts API

In [9]:
steam_04_jar_path = "/data/master_volume/map_reduce_jars/steam_04_fetch.jar"
steam_04_input_path = get_data_from_output_path(steam_03_output_path)
steam_04_output_path = "/out_steam_4" + str(uuid.uuid4())

res = run_in_master(f"yarn jar {steam_04_jar_path} {steam_04_input_path} {steam_04_output_path}")
print(res.exit_code == 0)

True


In [10]:
print_hdfs_output(steam_04_output_path)

242760	List([{"timestamp":1398902400000,"count":5722},{"timestamp":1401580800000,"count":6673},{"timestamp":1404172800000,"count":4725},{"timestamp":1406851200000,"count":3121},{"timestamp":1409529600000,"count":1875},{"timestamp":1412121600000,"count":2918},{"timestamp":1414800000000,"count":4640},{"timestamp":1417392000000,"count":3891},{"timestamp":1420070400000,"count":4809},{"timestamp":1422748800000,"count":3215},{"timestamp":1425168000000,"count":3194},{"timestamp":1427846400000,"count":2241},{"timestamp":1430438400000,"count":1990},{"timestamp":1433116800000,"count":4689},{"timestamp":1435708800000,"count":3421},{"timestamp":1438387200000,"count":4060},{"timestamp":1441065600000,"count":3686},{"timestamp":1443657600000,"count":3240},{"timestamp":1446336000000,"count":3788},{"timestamp":1448928000000,"count":4962},{"timestamp":1451606400000,"count":6245},{"timestamp":1454284800000,"count":6306},{"timestamp":1456790400000,"count":4793},{"timestamp":1459468800000,"count":4767},{"t